# Intense Experience API - Complete Data Fetch

This notebook fetches all available data from the Mews API and extracts all IDs for all objects.


In [1]:
# Import required libraries
import requests
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Mews API Configuration
MEWS_BASE_URL = "https://api.mews-demo.com/api/connector/v1"
CLIENT_TOKEN = os.getenv('ClientToken', 'E0D439EE522F44368DC78E1BFB03710C-D24FB11DBE31D4621C4817E028D9E1D')
ACCESS_TOKEN = os.getenv('AccessToken', 'C618020DC2C24A6DAEF7B38A012C43C1-246E13D8FC45C4A80E61D412555A97E')
ENTERPRISE_ID = "c390a691-e9a0-4aa0-860c-b3850108ab4c"

# Service IDs
ALL_SERVICE_IDS = [
    "05644daa-3d55-452c-b434-b3850108bbc3",  # Boisson & Nourriture
    "f4137fc1-1b72-40d9-97e2-b3850108bbc3",  # Services Divers
    "a094b98c-b6fd-4936-a9c7-b3850108bbc3",  # Packages
    "56d5f465-a1da-4b62-a9fb-b3850108bbc3",  # Bon cadeau
    "fb46039c-9c3f-4eb6-ba31-b3850108bbc3",  # Services suppl.
    "86fcc6a7-75ce-457a-a425-b3850108b6bf",  # JOURNEE
    "34e0d02a-36d2-4bf5-82a7-b3850108b729",  # DAYUSE
    "7ba0b732-93cc-477a-861d-b3850108b730",  # NUITEE
]

print("Configuration loaded successfully!")
print(f"Enterprise ID: {ENTERPRISE_ID}")
print(f"Number of Service IDs: {len(ALL_SERVICE_IDS)}")


Configuration loaded successfully!
Enterprise ID: c390a691-e9a0-4aa0-860c-b3850108ab4c
Number of Service IDs: 8


In [2]:
# Helper function to make API requests
def make_mews_request(endpoint, payload):
    """Make a request to Mews API"""
    url = f"{MEWS_BASE_URL}/{endpoint}"
    payload.update({
        "ClientToken": CLIENT_TOKEN,
        "AccessToken": ACCESS_TOKEN
    })
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        print(f"Response: {response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return None

# Helper function to extract IDs from a list of objects
def extract_ids(data_list, id_field='Id'):
    """Extract IDs from a list of objects"""
    if not data_list:
        return []
    return [item.get(id_field) for item in data_list if item.get(id_field)]

# Helper function to pretty print JSON
def pretty_print(data, title=""):
    """Pretty print JSON data"""
    if title:
        print(f"\n{'='*80}")
        print(f"{title}")
        print('='*80)
    print(json.dumps(data, indent=2))

print("Helper functions defined!")


Helper functions defined!


## 1. Fetch Configuration


In [3]:
# 1. Get Configuration
print("Fetching Configuration...")
configuration = make_mews_request("configuration/get", {})

if configuration:
    print("✓ Configuration fetched successfully")
    
    # Extract IDs from configuration
    config_ids = {
        "enterprise_id": configuration.get("Enterprise", {}).get("Id"),
        "enterprise_address_id": configuration.get("Enterprise", {}).get("AddressId"),
        "chain_id": configuration.get("Enterprise", {}).get("ChainId"),
    }
    
    print(f"\nConfiguration IDs:")
    for key, value in config_ids.items():
        print(f"  {key}: {value}")
else:
    print("✗ Failed to fetch configuration")
    config_ids = {}


Fetching Configuration...
✓ Configuration fetched successfully

Configuration IDs:
  enterprise_id: c390a691-e9a0-4aa0-860c-b3850108ab4c
  enterprise_address_id: 4fd32111-edfe-4a44-93d1-b3850108cc87
  chain_id: a02d6296-9d65-42af-a8ba-b1e100dc2b78


## 2. Fetch All Services


In [4]:
# 2. Get All Services
print("Fetching Services...")
services_result = make_mews_request("services/getAll", {})

if services_result and "Services" in services_result:
    services = services_result["Services"]
    print(f"✓ Fetched {len(services)} services")
    
    service_ids = extract_ids(services)
    
    print(f"\nService IDs and Names:")
    for service in services:
        print(f"  {service.get('Id')}: {service.get('Name')} ({service.get('Type')})")
    
    # Store for later use
    all_data = {"services": services}
else:
    print("✗ Failed to fetch services")
    service_ids = []
    all_data = {}


Fetching Services...
✓ Fetched 8 services

Service IDs and Names:
  05644daa-3d55-452c-b434-b3850108bbc3: Boisson & Nourriture (Orderable)
  f4137fc1-1b72-40d9-97e2-b3850108bbc3: Services Divers (Orderable)
  a094b98c-b6fd-4936-a9c7-b3850108bbc3: Packages (Orderable)
  56d5f465-a1da-4b62-a9fb-b3850108bbc3: Bon cadeau (Orderable)
  fb46039c-9c3f-4eb6-ba31-b3850108bbc3: Services suppl. (Orderable)
  86fcc6a7-75ce-457a-a425-b3850108b6bf: JOURNEE (Reservable)
  34e0d02a-36d2-4bf5-82a7-b3850108b729: DAYUSE (Reservable)
  7ba0b732-93cc-477a-861d-b3850108b730: NUITEE (Reservable)


## 3. Fetch All Resources


In [5]:
# 3. Get All Resources
print("Fetching Resources...")
resources_result = make_mews_request("resources/getAll", {})

if resources_result and "Resources" in resources_result:
    resources = resources_result["Resources"]
    print(f"✓ Fetched {len(resources)} resources")
    
    resource_ids = extract_ids(resources)
    
    print(f"\nResource IDs:")
    for resource in resources[:10]:  # Show first 10
        print(f"  {resource.get('Id')}: {resource.get('Name')}")
    if len(resources) > 10:
        print(f"  ... and {len(resources) - 10} more")
    
    all_data["resources"] = resources
else:
    print("✗ Failed to fetch resources")
    resource_ids = []


Fetching Resources...
✓ Fetched 11 resources

Resource IDs:
  300edef0-8e93-462c-9490-b3850108c67c: 101
  108ba0e3-85f8-446f-ac81-b3850108c67c: 201
  c0745b82-a8ae-4d87-bfbf-b3850108c67c: 202
  39d42002-7455-4ded-847a-b3850108c67c: 203
  1ad43a6f-2077-4231-861f-b3850108c67c: 204
  cf465ce7-3fc4-4907-b20d-b3850108c67d: 301
  e4759bfe-c119-49c3-824a-b3850108c67d: 302
  037ce22e-0f51-440a-a304-b3850108c67d: 303
  c64f5781-e5cf-40e4-979c-b3850108c67c: 401
  aac871a0-a5b3-41d4-a522-b3850108c67c: Batiment
  ... and 1 more


## 4. Fetch All Resource Categories (Rooms/Suites)


In [6]:
# 4. Get All Resource Categories (for all services)
print("Fetching Resource Categories...")
resource_categories_payload = {
    "EnterpriseIds": [ENTERPRISE_ID],
    "ServiceIds": ALL_SERVICE_IDS,
    "IncludeDefault": False,
    "Limitation": {"Count": 100}
}

resource_categories_result = make_mews_request("resourceCategories/getAll", resource_categories_payload)

if resource_categories_result and "ResourceCategories" in resource_categories_result:
    resource_categories = resource_categories_result["ResourceCategories"]
    print(f"✓ Fetched {len(resource_categories)} resource categories")
    
    resource_category_ids = extract_ids(resource_categories)
    
    print(f"\nResource Category IDs and Names:")
    for category in resource_categories:
        print(f"  {category.get('Id')}: {category.get('Name')} ({category.get('Type')})")
    
    all_data["resource_categories"] = resource_categories
else:
    print("✗ Failed to fetch resource categories")
    resource_category_ids = []


Fetching Resource Categories...
✓ Fetched 24 resource categories

Resource Category IDs and Names:
  6be75ead-2ed0-44b1-bb78-b3850108c66f: None (Room)
  f723bd5a-04fe-479c-bab4-b3850108c66f: None (Suite)
  f9c359dc-7457-43b8-b914-b3850108c66f: None (Suite)
  f5539e51-9db0-4082-b87e-b3850108c66f: None (Room)
  68b87fe5-7b78-4067-b5e7-b3850108c66f: None (Suite)
  c1561e97-953a-4461-b279-b3850108c66f: None (Room)
  3872869e-6278-4c64-aea8-b3850108c66f: None (Suite)
  78a614b1-199d-4608-ab89-b3850108c66f: None (Room)
  3736a5c4-9214-4d5a-a619-b3850108c66f: None (PrivateSpaces)
  ef005151-3afa-4a15-a504-b3850108c66f: None (Room)
  e4706d3a-2a06-4cb7-a449-b3850108c66f: None (Suite)
  d8671c5c-33f1-4ca0-a1fe-b3850108c66f: None (Room)
  9e9feb39-c947-4703-9c9a-b3850108c66f: None (PrivateSpaces)
  d6ed9517-b955-4dec-9c82-b3850108c66f: None (PrivateSpaces)
  2b3a2b96-5d61-49df-9af1-b3850108c66f: None (Suite)
  f867b5c6-f62d-451c-96ec-b3850108c66f: None (Suite)
  6d70bbb6-ce34-435c-95f6-b3850108c

## 5. Fetch All Rates


In [7]:
# 5. Get All Rates
print("Fetching Rates...")
rates_payload = {
    "EnterpriseIds": [ENTERPRISE_ID],
    "IncludeDefault": False,
    "Limitation": {"Count": 100}
}

rates_result = make_mews_request("rates/getAll", rates_payload)

if rates_result and "Rates" in rates_result:
    rates = rates_result["Rates"]
    print(f"✓ Fetched {len(rates)} rates")
    
    rate_ids = extract_ids(rates)
    
    print(f"\nRate IDs and Names:")
    for rate in rates:
        print(f"  {rate.get('Id')}: {rate.get('Name')}")
    
    all_data["rates"] = rates
else:
    print("✗ Failed to fetch rates")
    rate_ids = []


Fetching Rates...
✓ Fetched 13 rates

Rate IDs and Names:
  5be89056-b99f-41bb-b5cb-b3850108c576: Partenaires
  2e3cba42-cd31-4649-b75b-b3850108c562: Tarif Interne Journée Semaine
  c8c7be05-b9d8-41fe-93a4-b3850108c54e: Tarif Interne Journée Weekend
  d0496fa0-6686-4614-8847-b3850108c537: TARIF JOURNEE LE WEEKEND
  197861bb-bc45-4920-b9d7-b3850108c523: Tarif Suites Nuitée Interne
  929d64ee-2aa4-448c-9a97-b3850108c50f: Promo Emailing
  6daf0d4e-b594-444b-8b63-b3850108c4fc: Partenaires
  efedeb6f-730b-42a2-8e7c-b3850108c4dc: Booking Non remboursable
  8e41d761-09d7-4485-b212-b3850108c4be: Tarif Booking
  05fd7d4f-7744-4213-99cf-b3850108c4a0: Tarif Airbnb
  2f2d2176-db9c-4a4e-ab44-b3850108b8b0: DAYUSE 
  ed9391ac-b184-4876-8cc1-b3850108b8b0: Tarif Suites nuitée
  c3c2109d-984a-4ad4-978e-b3850108b8ad: TARIF JOURNEE EN SEMAINE


## 6. Fetch All Products


In [8]:
# 6. Get All Products
print("Fetching Products...")
products_payload = {
    "ServiceIds": ALL_SERVICE_IDS,
    "EnterpriseIds": [ENTERPRISE_ID],
    "IncludeDefault": False,
    "Limitation": {"Count": 100}
}

products_result = make_mews_request("products/getAll", products_payload)

if products_result and "Products" in products_result:
    products = products_result["Products"]
    print(f"✓ Fetched {len(products)} products")
    
    product_ids = extract_ids(products)
    
    print(f"\nProduct IDs and Names:")
    for product in products[:20]:  # Show first 20
        print(f"  {product.get('Id')}: {product.get('Name')}")
    if len(products) > 20:
        print(f"  ... and {len(products) - 20} more")
    
    all_data["products"] = products
else:
    print("✗ Failed to fetch products")
    product_ids = []


Fetching Products...
✓ Fetched 46 products

Product IDs and Names:
  55fd4ae6-76f4-4e32-bfbc-b3850108bdc1: Départ Tardif
  fdbfd35d-254c-4399-b5fb-b3850108bdc1: Arrivée anticipée
  ce65843a-52f2-4ccc-bd85-b3850108bcd9: Taxe de séjour
  3fd79992-c888-48a4-bd4b-b3850108bcd9: Vin
  38d6bba1-6634-462a-bbb3-b3850108bcd9: Serviettes de bain
  e21b9771-4270-403f-ba57-b3850108bcd9: Massage 60min
  a7583228-4519-48ea-ba4e-b3850108bcd9: Late ckeckout 50
  915d9715-5599-42b9-b937-b3850108bcd9: Champagne et Gourmandises
  fb79ba72-36bc-4604-b7e7-b3850108bcd9: Cocktail de Bienvenue
  a707f411-4dd5-4799-b440-b3850108bcd9: Early checkin 50
  50ee2c6f-8a8a-44ad-b17e-b3850108bcd9: Cocktail de Bienvenue
  234be69d-0e86-4555-aee5-b3850108bcd9: Achat Bon Cadeau
  9bde06bb-c3b4-4be8-ae2c-b3850108bcd9: Départ tardif
  45a97019-22d1-4dc5-adc8-b3850108bcd9: Massage Duo 60min
  fd98cb8e-9174-46ef-aae5-b3850108bcd9: Brunch
  2592a4b1-7080-48ec-aa72-b3850108bcd9: Massage Duo 1h
  06c29a1d-81ea-4203-aa1f-b3850108

## 7. Fetch All Age Categories


In [9]:
# 7. Get All Age Categories
print("Fetching Age Categories...")
age_categories_payload = {
    "EnterpriseIds": [ENTERPRISE_ID]
}

age_categories_result = make_mews_request("ageCategories/getAll", age_categories_payload)

if age_categories_result and "AgeCategories" in age_categories_result:
    age_categories = age_categories_result["AgeCategories"]
    print(f"✓ Fetched {len(age_categories)} age categories")
    
    age_category_ids = extract_ids(age_categories)
    
    print(f"\nAge Category IDs:")
    for category in age_categories:
        service_name = category.get('Names', {}).get('en-US', 'Unknown')
        print(f"  {category.get('Id')}: {service_name} (Service: {category.get('ServiceId')})")
    
    all_data["age_categories"] = age_categories
else:
    print("✗ Failed to fetch age categories")
    age_category_ids = []


Fetching Age Categories...
✓ Fetched 6 age categories

Age Category IDs:
  a78b7aca-fa0b-4199-8b4e-b3850108b8a5: Adults (Service: 86fcc6a7-75ce-457a-a425-b3850108b6bf)
  d260c711-e588-4716-a831-b3850108b8a5: Children (Service: 86fcc6a7-75ce-457a-a425-b3850108b6bf)
  a8777ef1-9cfc-417c-834e-b3850108b8a6: Adults (Service: 34e0d02a-36d2-4bf5-82a7-b3850108b729)
  8f962d74-7e5e-4d00-8c12-b3850108b8a6: Children (Service: 34e0d02a-36d2-4bf5-82a7-b3850108b729)
  6cef9c83-4199-4b40-972b-b3850108b8a6: Adults (Service: 7ba0b732-93cc-477a-861d-b3850108b730)
  1e3ab8bd-6292-4f22-a032-b3850108b8a6: Children (Service: 7ba0b732-93cc-477a-861d-b3850108b730)


## 8. Fetch All Customers


In [17]:
# 8. Get All Customers (with pagination support)
print("Fetching Customers...")

# Calculate date range for UpdatedUtc filter (last 90 days to stay within 3M1D limit)
now = datetime.utcnow()
updated_start = (now - timedelta(days=90)).isoformat() + 'Z'
updated_end = now.isoformat() + 'Z'

customers_payload = {
    "UpdatedUtc": {
        "StartUtc": updated_start,
        "EndUtc": updated_end
    },
    "Limitation": {"Count": 100}
}

customers_result = make_mews_request("customers/getAll", customers_payload)

if customers_result and "Customers" in customers_result:
    customers = customers_result["Customers"]
    print(f"✓ Fetched {len(customers)} customers")
    
    customer_ids = extract_ids(customers)
    
    print(f"\nCustomer IDs (first 10):")
    for customer in customers[:10]:
        print(f"  {customer.get('Id')}: {customer.get('FirstName', '')} {customer.get('LastName', '')}")
    if len(customers) > 10:
        print(f"  ... and {len(customers) - 10} more")
    
    all_data["customers"] = customers
else:
    print("✗ Failed to fetch customers")
    customer_ids = []


Fetching Customers...
✓ Fetched 7 customers

Customer IDs (first 10):
  9b315cd4-af33-4571-91c9-b396012c8f88: tyuugikhj ygkj
  e56eac16-319c-48d7-88a6-b396012bbed7: ighdfo doffjgdiof
  a5e59ad8-177a-456d-a6b7-b396010875d0: John Doe
  8ddfdad5-f6d3-41af-9f65-b3900110d08f: John Doe
  b6b5cc6b-ebc1-45a8-bc6f-b390010126cb: Louis Dresse
  11e2feb0-dc2b-410c-a058-b38c01157890: None Test 2
  0ca3f3a5-8273-436e-8af9-b38a011a7b8a: None Test


## 9. Fetch All Reservations


In [18]:
# 9. Get All Reservations (for next 2 months, in chunks to avoid 100 hour limit)
print("Fetching Reservations...")

# Calculate date range for next 2 months
now = datetime.utcnow()
end_date = now + timedelta(days=60)

# Split into chunks of 4 days (96 hours) to stay under 100 hour limit
all_reservations = []
current_start = now

while current_start < end_date:
    current_end = min(current_start + timedelta(days=4), end_date)
    
    start_date_str = current_start.isoformat() + 'Z'
    end_date_str = current_end.isoformat() + 'Z'
    
    print(f"  Fetching chunk: {start_date_str[:10]} to {end_date_str[:10]}...")
    
    reservations_payload = {
        "Client": "Intense Experience Testing",
        "StartUtc": start_date_str,
        "EndUtc": end_date_str
    }
    
    reservations_result = make_mews_request("reservations/getAll", reservations_payload)
    
    if reservations_result and "Reservations" in reservations_result:
        chunk_reservations = reservations_result["Reservations"]
        all_reservations.extend(chunk_reservations)
        print(f"    ✓ Fetched {len(chunk_reservations)} reservations in this chunk")
    else:
        print(f"    ✗ Failed to fetch reservations for this chunk")
    
    current_start = current_end

if all_reservations:
    reservations = all_reservations
    print(f"\n✓ Total fetched: {len(reservations)} reservations")
    
    reservation_ids = extract_ids(reservations)
    
    print(f"\nReservation IDs (first 10):")
    for reservation in reservations[:10]:
        print(f"  {reservation.get('Id')}: {reservation.get('State')} - {reservation.get('StartUtc')} to {reservation.get('EndUtc')}")
    if len(reservations) > 10:
        print(f"  ... and {len(reservations) - 10} more")
    
    all_data["reservations"] = reservations
else:
    print("✗ Failed to fetch reservations")
    reservation_ids = []


Fetching Reservations...
  Fetching chunk: 2025-11-16 to 2025-11-20...
    ✓ Fetched 0 reservations in this chunk
  Fetching chunk: 2025-11-20 to 2025-11-24...
    ✓ Fetched 2 reservations in this chunk
  Fetching chunk: 2025-11-24 to 2025-11-28...
    ✓ Fetched 0 reservations in this chunk
  Fetching chunk: 2025-11-28 to 2025-12-02...
    ✓ Fetched 0 reservations in this chunk
  Fetching chunk: 2025-12-02 to 2025-12-06...
    ✓ Fetched 0 reservations in this chunk
  Fetching chunk: 2025-12-06 to 2025-12-10...
    ✓ Fetched 0 reservations in this chunk
  Fetching chunk: 2025-12-10 to 2025-12-14...
    ✓ Fetched 0 reservations in this chunk
  Fetching chunk: 2025-12-14 to 2025-12-18...
    ✓ Fetched 0 reservations in this chunk
  Fetching chunk: 2025-12-18 to 2025-12-22...
    ✓ Fetched 0 reservations in this chunk
  Fetching chunk: 2025-12-22 to 2025-12-26...
    ✓ Fetched 0 reservations in this chunk
  Fetching chunk: 2025-12-26 to 2025-12-30...
    ✓ Fetched 0 reservations in this ch

## 10. Fetch Rate Pricing (for a sample rate and date range)


In [21]:
# 10. Get Rate Pricing (if we have rates)
if rate_ids and len(rate_ids) > 0:
    print("Fetching Rate Pricing for first rate...")
    
    # Use first rate
    sample_rate_id = rate_ids[0]
    
    # Next 7 days at 23:00:00.000Z
    first_date = datetime.utcnow().replace(hour=23, minute=0, second=0, microsecond=0).isoformat() + 'Z'
    last_date = (datetime.utcnow() + timedelta(days=7)).replace(hour=23, minute=0, second=0, microsecond=0).isoformat() + 'Z'
    
    pricing_payload = {
        "EnterpriseIds": [ENTERPRISE_ID],
        "Client": "Intense Experience Testing",
        "RateId": sample_rate_id,
        "FirstTimeUnitStartUtc": first_date,
        "LastTimeUnitStartUtc": last_date
    }
    
    pricing_result = make_mews_request("rates/getPricing", pricing_payload)
    
    if pricing_result and "CategoryPrices" in pricing_result:
        category_prices = pricing_result["CategoryPrices"]
        print(f"✓ Fetched pricing for {len(category_prices)} categories")
        
        print(f"\nPricing for Rate {sample_rate_id}:")
        for cat_price in category_prices:  # Show all
            category_id = cat_price.get('CategoryId', 'N/A')
            price_info = cat_price.get('Price', {})
            gross_value = price_info.get('GrossValue', 'N/A')
            currency = pricing_result.get('Currency', 'EUR')
            print(f"  Category {category_id}: {gross_value} {currency}")
        
        all_data["rate_pricing"] = pricing_result
    else:
        print("✗ Failed to fetch rate pricing")
else:
    print("Skipping rate pricing (no rates available)")


Fetching Rate Pricing for first rate...
✓ Fetched pricing for 11 categories

Pricing for Rate 5be89056-b99f-41bb-b5cb-b3850108c576:
  Category 32064033-7392-4c44-8c70-b3850108c66f: N/A EUR
  Category 1113bcbe-ad5f-49c7-8dc0-b3850108c66f: N/A EUR
  Category 67ece5a2-65e2-43c5-9079-b3850108c66f: N/A EUR
  Category 90df19cc-469e-40b2-94ad-b3850108c66f: N/A EUR
  Category 9e9feb39-c947-4703-9c9a-b3850108c66f: N/A EUR
  Category e4706d3a-2a06-4cb7-a449-b3850108c66f: N/A EUR
  Category 3736a5c4-9214-4d5a-a619-b3850108c66f: N/A EUR
  Category 78a614b1-199d-4608-ab89-b3850108c66f: N/A EUR
  Category 68b87fe5-7b78-4067-b5e7-b3850108c66f: N/A EUR
  Category f5539e51-9db0-4082-b87e-b3850108c66f: N/A EUR
  Category f723bd5a-04fe-479c-bab4-b3850108c66f: N/A EUR


## Summary: All IDs Collected


In [23]:
# Compile all IDs in one summary
print("\n" + "="*80)
print("SUMMARY OF ALL IDs")
print("="*80)

all_ids = {
    "configuration": config_ids,
    "services": {
        "ids": service_ids,
        "count": len(service_ids)
    },
    "resources": {
        "ids": resource_ids if 'resource_ids' in locals() else [],
        "count": len(resource_ids) if 'resource_ids' in locals() else 0
    },
    "resource_categories": {
        "ids": resource_category_ids if 'resource_category_ids' in locals() else [],
        "count": len(resource_category_ids) if 'resource_category_ids' in locals() else 0
    },
    "rates": {
        "ids": rate_ids if 'rate_ids' in locals() else [],
        "count": len(rate_ids) if 'rate_ids' in locals() else 0
    },
    "products": {
        "ids": product_ids if 'product_ids' in locals() else [],
        "count": len(product_ids) if 'product_ids' in locals() else 0
    },
    "age_categories": {
        "ids": age_category_ids if 'age_category_ids' in locals() else [],
        "count": len(age_category_ids) if 'age_category_ids' in locals() else 0
    },
    "customers": {
        "ids": customer_ids if 'customer_ids' in locals() else [],
        "count": len(customer_ids) if 'customer_ids' in locals() else 0
    },
    "reservations": {
        "ids": reservation_ids if 'reservation_ids' in locals() else [],
        "count": len(reservation_ids) if 'reservation_ids' in locals() else 0
    }
}

# Print summary
for category, data in all_ids.items():
    if category == "configuration":
        print(f"\n{category.upper()}:")
        for key, value in data.items():
            print(f"  {key}: {value}")
    else:
        print(f"\n{category.upper()}: {data['count']} items")

print("\n" + "="*80)



SUMMARY OF ALL IDs

CONFIGURATION:
  enterprise_id: c390a691-e9a0-4aa0-860c-b3850108ab4c
  enterprise_address_id: 4fd32111-edfe-4a44-93d1-b3850108cc87
  chain_id: a02d6296-9d65-42af-a8ba-b1e100dc2b78

SERVICES: 8 items

RESOURCES: 11 items

RESOURCE_CATEGORIES: 24 items

RATES: 13 items

PRODUCTS: 46 items

AGE_CATEGORIES: 6 items

CUSTOMERS: 7 items

RESERVATIONS: 2 items



## Export All Data to JSON File


In [24]:
# Export all data to a JSON file
output_filename = "mews_api_data.json"

export_data = {
    "fetched_at": datetime.utcnow().isoformat() + 'Z',
    "enterprise_id": ENTERPRISE_ID,
    "summary": {
        "total_services": len(service_ids) if 'service_ids' in locals() else 0,
        "total_resources": len(resource_ids) if 'resource_ids' in locals() else 0,
        "total_resource_categories": len(resource_category_ids) if 'resource_category_ids' in locals() else 0,
        "total_rates": len(rate_ids) if 'rate_ids' in locals() else 0,
        "total_products": len(product_ids) if 'product_ids' in locals() else 0,
        "total_age_categories": len(age_category_ids) if 'age_category_ids' in locals() else 0,
        "total_customers": len(customer_ids) if 'customer_ids' in locals() else 0,
        "total_reservations": len(reservation_ids) if 'reservation_ids' in locals() else 0,
    },
    "all_ids": all_ids,
    "full_data": all_data
}

# Save to file
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(export_data, f, indent=2, ensure_ascii=False)

print(f"\n✓ All data exported to: {output_filename}")
print(f"  File size: {os.path.getsize(output_filename) / 1024:.2f} KB")



✓ All data exported to: mews_api_data.json
  File size: 1330.79 KB


## Quick Reference: Display All IDs in Clean Format


In [26]:
# Display all IDs in a clean, organized format
print("\n" + "="*80)
print("COMPLETE ID REFERENCE")
print("="*80)

print("\n📋 CONFIGURATION IDs:")
print("-" * 80)
if config_ids:
    for key, value in config_ids.items():
        print(f"  {key:30s} : {value}")

if 'service_ids' in locals() and service_ids:
    print("\n🔧 SERVICE IDs:")
    print("-" * 80)
    for service in all_data.get('services', []):
        service_id = service.get('Id') or 'N/A'
        service_name = service.get('Name') or 'N/A'
        service_type = service.get('Type') or 'N/A'
        print(f"  {service_id:36s} | {service_name:20s} | Type: {service_type}")

if 'resource_ids' in locals() and resource_ids:
    print(f"\n🏢 RESOURCE IDs: ({len(resource_ids)} total)")
    print("-" * 80)
    for resource in all_data.get('resources', [])[:20]:
        resource_id = resource.get('Id') or 'N/A'
        resource_name = resource.get('Name') or 'N/A'
        print(f"  {resource_id:36s} | {resource_name}")
    if len(resource_ids) > 20:
        print(f"  ... and {len(resource_ids) - 20} more")

if 'resource_category_ids' in locals() and resource_category_ids:
    print(f"\n🛏️  RESOURCE CATEGORY IDs (Suites/Rooms):")
    print("-" * 80)
    for category in all_data.get('resource_categories', []):
        category_id = category.get('Id') or 'N/A'
        category_name = category.get('Name') or 'N/A'
        category_type = category.get('Type') or 'N/A'
        print(f"  {category_id:36s} | {category_name:20s} | Type: {category_type}")

if 'rate_ids' in locals() and rate_ids:
    print(f"\n💰 RATE IDs:")
    print("-" * 80)
    for rate in all_data.get('rates', []):
        rate_id = rate.get('Id') or 'N/A'
        rate_name = rate.get('Name') or 'N/A'
        print(f"  {rate_id:36s} | {rate_name}")

if 'product_ids' in locals() and product_ids:
    print(f"\n🛍️  PRODUCT IDs: ({len(product_ids)} total)")
    print("-" * 80)
    for product in all_data.get('products', [])[:15]:
        product_id = product.get('Id') or 'N/A'
        product_name = product.get('Name') or 'N/A'
        print(f"  {product_id:36s} | {product_name}")
    if len(product_ids) > 15:
        print(f"  ... and {len(product_ids) - 15} more")

if 'age_category_ids' in locals() and age_category_ids:
    print(f"\n👥 AGE CATEGORY IDs:")
    print("-" * 80)
    for category in all_data.get('age_categories', []):
        category_id = category.get('Id') or 'N/A'
        name = category.get('Names', {}).get('en-US', 'Unknown')
        service_id = category.get('ServiceId') or 'N/A'
        print(f"  {category_id:36s} | {name:15s} | Service: {service_id}")

if 'customer_ids' in locals() and customer_ids:
    print(f"\n👤 CUSTOMER IDs: ({len(customer_ids)} total - showing first 10)")
    print("-" * 80)
    for customer in all_data.get('customers', [])[:10]:
        customer_id = customer.get('Id') or 'N/A'
        name = f"{customer.get('FirstName', '')} {customer.get('LastName', '')}"
        print(f"  {customer_id:36s} | {name}")
    if len(customer_ids) > 10:
        print(f"  ... and {len(customer_ids) - 10} more")

if 'reservation_ids' in locals() and reservation_ids:
    print(f"\n📅 RESERVATION IDs: ({len(reservation_ids)} total - showing first 10)")
    print("-" * 80)
    for reservation in all_data.get('reservations', [])[:10]:
        reservation_id = reservation.get('Id') or 'N/A'
        state = reservation.get('State') or 'N/A'
        start_date = reservation.get('StartUtc', '')[:10] if reservation.get('StartUtc') else 'N/A'
        print(f"  {reservation_id:36s} | State: {state:12s} | {start_date}")
    if len(reservation_ids) > 10:
        print(f"  ... and {len(reservation_ids) - 10} more")

print("\n" + "="*80)
print("END OF ID REFERENCE")
print("="*80)



COMPLETE ID REFERENCE

📋 CONFIGURATION IDs:
--------------------------------------------------------------------------------
  enterprise_id                  : c390a691-e9a0-4aa0-860c-b3850108ab4c
  enterprise_address_id          : 4fd32111-edfe-4a44-93d1-b3850108cc87
  chain_id                       : a02d6296-9d65-42af-a8ba-b1e100dc2b78

🔧 SERVICE IDs:
--------------------------------------------------------------------------------
  05644daa-3d55-452c-b434-b3850108bbc3 | Boisson & Nourriture | Type: Orderable
  f4137fc1-1b72-40d9-97e2-b3850108bbc3 | Services Divers      | Type: Orderable
  a094b98c-b6fd-4936-a9c7-b3850108bbc3 | Packages             | Type: Orderable
  56d5f465-a1da-4b62-a9fb-b3850108bbc3 | Bon cadeau           | Type: Orderable
  fb46039c-9c3f-4eb6-ba31-b3850108bbc3 | Services suppl.      | Type: Orderable
  86fcc6a7-75ce-457a-a425-b3850108b6bf | JOURNEE              | Type: Reservable
  34e0d02a-36d2-4bf5-82a7-b3850108b729 | DAYUSE               | Type: Reservable

## Export IDs to Separate CSV Files (Optional)


## Final Notes

This notebook has fetched all available data from the following Mews API endpoints:

1. **Configuration** - Enterprise and system configuration
2. **Services** - All service types (Reservable and Orderable)
3. **Resources** - Physical resources (rooms, spaces, etc.)
4. **Resource Categories** - Categories of rooms/suites
5. **Rates** - Pricing rates
6. **Products** - Additional products and services
7. **Age Categories** - Age classifications for pricing
8. **Customers** - Customer database (paginated)
9. **Reservations** - Recent and upcoming reservations
10. **Rate Pricing** - Sample pricing data for a rate

### Output Files:
- **JSON file**: Complete data dump with all IDs and full objects
- **CSV files**: Individual CSV files for each category for easy reference

### Important IDs:
- Enterprise ID: `c390a691-e9a0-4aa0-860c-b3850108ab4c`
- JOURNEE Service ID: `86fcc6a7-75ce-457a-a425-b3850108b6bf`
- NUITEE Service ID: `7ba0b732-93cc-477a-861d-b3850108b730`

You can now use these IDs in your application for testing and development!


## Quick Access: Get Specific IDs Programmatically


In [ ]:
# Helper functions to quickly find specific IDs by name or other criteria

def find_service_by_name(name):
    """Find service ID by name"""
    for service in all_data.get('services', []):
        if name.lower() in service.get('Name', '').lower():
            return service
    return None

def find_rate_by_name(name):
    """Find rate ID by name"""
    for rate in all_data.get('rates', []):
        if name.lower() in rate.get('Name', '').lower():
            return rate
    return None

def find_resource_category_by_name(name):
    """Find resource category ID by name"""
    for category in all_data.get('resource_categories', []):
        if name.lower() in category.get('Name', '').lower():
            return category
    return None

def find_product_by_name(name):
    """Find product ID by name"""
    for product in all_data.get('products', []):
        if name.lower() in product.get('Name', '').lower():
            return product
    return None

# Example usage:
print("Example searches:")
print("-" * 80)

# Find NUITEE service
nuitee = find_service_by_name("NUITEE")
if nuitee:
    print(f"NUITEE Service ID: {nuitee.get('Id')}")

# Find JOURNEE service
journee = find_service_by_name("JOURNEE")
if journee:
    print(f"JOURNEE Service ID: {journee.get('Id')}")

# List all resource categories
print(f"\nAll Resource Categories:")
for cat in all_data.get('resource_categories', []):
    print(f"  - {cat.get('Name'):20s} (ID: {cat.get('Id')})")

print("\n" + "="*80)
